In [14]:
import torch
import torch.nn as nn
import nltk
from nltk.corpus import gutenberg
import gensim
import json

### Get all sentences in Project Gutenberg

In [2]:
sents = []
for file in gutenberg.fileids():
    for sent in gutenberg.sents(file):
        sents.append(sent)
len(sents)

98552

### Train a word2vec model using gensim for comparison

In [3]:
model = gensim.models.Word2Vec(sents, size=100, window=5, min_count=1, workers=4)
model.save('word2vec_gensim')

### Create Vocabulary for all the words in Project Gutenberg

In [4]:
words = []
for file in gutenberg.fileids():
    for word in gutenberg.words(file):
        words.append(word)
# words = list(set(words))
len(words)

2621613

In [5]:
fd = nltk.FreqDist(words)
vocab = sorted(fd, key=fd.get, reverse=True)
vocab[:10]

[',', 'the', 'and', '.', 'of', ':', 'to', 'a', 'in', 'I']

In [6]:
with open('vocab', 'wb') as f:
    for i in range(len(vocab)):
        f.write('{} {}\n'.format(vocab[i], i).encode('utf-8'))

###### Since we only have 2.6 million words, Skip-Gram should performs better

### Set hyperparameters 

In [8]:
settings = {
    'window_size': 5,
    'num_epochs': 100,
    'embedding_dim': 50,
    'learning_rate': 1e-5
}

In [46]:
sents[4][max(0, 4-2):4], sents[4][37+1:min(len(sents[4]), 37+3)]

(['the', 'youngest'], ['.'])

In [42]:
sents[4][4+1:min(39, 7)]

['the', 'two']

In [45]:
sents[4].index('period')

37

In [61]:
from torch.utils.data.dataset import Dataset
class myDataset(Dataset):
    
    def __init__(self, settings):
        self.window_size = settings['window_size']
        self.dim = settings['embedding_dim']
        # read from project gutenberg
        sents = []
        list(map(sents.extend, list(map(gutenberg.sents, gutenberg.fileids()))))
        print('\n{} sentences fetched.'.format(len(sents)))
        # load vocabulary file
        with open('vocab.json', 'r') as f:
            vocab = json.load(f)
        print('\n{} words found in corpus'.format(len(vocab)))
        self.word2id = dict((vocab[i], i) for i in range(len(vocab)))
        self.data = []
        for sent in sents:
            for i in range(len(sent)):
                context = [self.word2id[word] for word in sent[max(0, i - self.window_size):i] + sent[i+1:max(
                    len(sent), i + 1 + self.window_size)]]
                target = self.word2id[sent[i]]
                self.data.append([target, context])
        
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        return self.data[index]

In [58]:
with open('vocab.json', 'w') as f:
    json.dump(vocab, f)

In [59]:
with open('vocab.json', 'r') as f:
    print(len(json.load(f)))

51156


In [62]:
dataset = myDataset(settings)


98552 sentences fetched.

51156 words found in corpus


KeyError: '"*'

In [52]:
datset.__len__()

NameError: name 'datset' is not defined

In [ ]:
class model(nn.Module):
    def __init__(self, num_hidden, num_dim):
        super(model, self).__init__()
        self.W_Q = nn.Linear(num_dim)